In [1]:
import json
from bs4 import Tag
from tqdm import tqdm
import tbwriters_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:
file_name_code = "tbwriters"

In [3]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

# def read_json(path, file_name):
#     try:
#         with open(path+file_name, 'r', encoding='utf-8') as openfile:
#             Loaded_file = json.load(openfile)
#             print(f"Successfully loaded: {file_name}")
#         return Loaded_file
#     except Exception as e:
#         print(f"Error loading {file_name}: {str(e)}")
#         return None

def scrape_article(url, page_key_code):
    try:
        article_content = tbwriters_utils.scrape_tbwriters_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None


def get_content(All_links_data, Total_length, page_key_code, page_key_list):
    all_article = {}
    save_interval = 10  # Save every 10 articles
    save_counter = 0
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_length+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_{file_name_code}_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
                    
                    save_counter += 1
                    if save_counter >= save_interval:
                        save_file_name = f"{file_name_code}_ALL_content_{page_key_list[1]}_partial.json"
                        path = "./data/parallel_content/"
                        save_json(path, save_file_name, all_article)
                        save_counter = 0
                        
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    # Save the final complete file
    save_file_name = f"{file_name_code}_ALL_content_{page_key_list[1]}.json"
    print(f"Saving final file: {save_file_name}")
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=11) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [ ]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in tbwriters_ALL_link_སྙན་ངག.json: 2
page key name: Page སྙན་ངག 2
Page key code: Page སྙན་ངག 
Total page in tbwriters_ALL_link_གདོང་ཤོག.json: 25
page key name: Page གདོང་ཤོག 25
Page key code: Page གདོང་ཤོག 
Total page in tbwriters_ALL_link_ཟིན་བྲིས།.json: 7
page key name: Page ཟིན་བྲིས། 7
Page key code: Page ཟིན་བྲིས། 
Total page in tbwriters_ALL_link_དཔྱད་རྩོམ།.json: 6
page key name: Page དཔྱད་རྩོམ། 6
Page key code: Page དཔྱད་རྩོམ། 
Total page in tbwriters_ALL_link_ལྷུག་རྩོམ།.json: 1
page key name: Page ལྷུག་རྩོམ། 1
Page key code: Page ལྷུག་རྩོམ། 
Total page in tbwriters_ALL_link_བསྒྱུར་རྩོམ།.json: 3
page key name: Page བསྒྱུར་རྩོམ། 3
Page key code: Page བསྒྱུར་རྩོམ། 
Total page in tbwriters_ALL_link_སྒྲུང་རྩོམ།.json: 2
page key name: Page སྒྲུང་རྩོམ། 2
Page key code: Page སྒྲུང་རྩོམ། 
Total page in tbwriters_ALL_link_ལོ་རྒྱུས།.json: 2
page key name: Page ལོ་རྒྱུས། 2
Page key code: Page ལོ་རྒྱུས། 
Total page in tbwriters_ALL_link_འབོལ་རྩོམ།.json: 1
page key name: Page འབོལ་





  0%|          | 0/4 [00:00<?, ?it/s]]


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]




  0%|          | 0/44 [00:00<?, ?it/s]





  0%|          | 0/85 [00:00<?, ?it/s]








  0%|          | 0/219 [00:00<?, ?it/s]









  0%|          | 0/251 [00:00<?, ?it/s]







  0%|          | 0/274 [00:00<?, ?it/s]






  0%|          | 0/63 [00:00<?, ?it/s]










  0%|          | 0/962 [00:00<?, ?it/s]










  0%|          | 1/962 [01:03<16:54:39, 63.35s/it]








  0%|          | 1/219 [01:11<4:21:12, 71.89s/it]

Soup content length: 87485










  0%|          | 1/274 [01:13<5:35:54, 73.83s/it]

Soup content length: 71703


...

Soup content length: 71495












 96%|█████████▌| 241/251 [1:05:01<03:31, 21.17s/it]

Soup content length: 73708












 96%|█████████▋| 242/251 [1:05:41<04:02, 26.90s/it]

Soup content length: 64072


In [5]:
# 14:44

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files

In [ ]:
len(json_files)